<table align="left">
  <tr>
    <td><img src="Museum.png" alt="ice cream truck in front of Louvre" width="120"/></td>
    <td align="left"><h1>Lesson 5: Building a Deep Research-like agent</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> :</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Set up and test search tools

In [1]:
from tavily import TavilyClient
from smolagents import tool, Tool
import os
from dotenv import load_dotenv

load_dotenv(override=True) # load variables from local .env file

@tool
def web_search(query: str) -> str:
    """Searches the web for your query.

    Args:
        query: Your query
    """
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    response = tavily_client.search(query)
    return str(response["results"])


class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def forward(self, url: str) -> str:
        try:
            import re

            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException

            from smolagents.utils import truncate_content
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, 40000)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

print(web_search("What is the melting temperature of vanilla ice cream in °C?"))

[{'title': "Hot, hot, hot! the ultimate guide to understanding ice cream's melting ...", 'url': 'https://delifo.net/why-ice-cream-melts/', 'content': '3. What is the ideal temperature to store ice cream? The ideal temperature to store ice cream is between -10°F (-23°C) and 0°F (-18°C). This temperature range helps to maintain the frozen state and prevent rapid melting.', 'score': 0.6485337, 'raw_content': None}, {'title': 'What Is the Melting Point of Ice Cream? - Reference.com', 'url': 'https://www.reference.com/science-technology/melting-point-ice-cream-d438071a06dd9e19', 'content': 'What Is the Melting Point of Ice Cream? According to the University of California – Santa Barbra, ice cream will melt at about 31 degrees Fahrenheit or -3 degrees Celsius. The freezing point of water is about 32 degrees Fahrenheit, but the ice cream is affected by the salt content. Adding salt to a liquid lowers the freezing point of the substance. The reason why salt affects the freezing point of liquid

## Set up an agent

In [2]:
from PIL import Image

from smolagents import CodeAgent, OpenAIServerModel

from huggingface_hub import login

login(os.getenv('HF_API_KEY'))

model = OpenAIServerModel(
    "gpt-4o",
    max_completion_tokens=8096,
)

<div style="font-size: 20px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> In the notebook, we reduced the museum count from 10 to 3 to reduce runtime and cost.  
  This will reduce the number of ice cream trucks you will need to buy to start your business!
</div>


In [3]:
request_museums = """
Could you give me a sorted list of the top 3 museums in the world in 2023,
along with their visitor count (in millions) that year, and the approximate daily temperature
in July at their location ?"""

In [ ]:
agent = CodeAgent(
    model=model,
    tools=[web_search, VisitWebpageTool()],
    max_steps=10
)
agent.logger.console.width=66
result = agent.run(request_museums)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Could you give me a sorted list of the top 3 museums in the    │
│ world in 2023,                                                 │
│ along with their visitor count (in millions) that year, and    │
│ the approximate daily temperature                              │
│ in July at their location ?                                    │
│                                                                │
╰─ OpenAIServerModel - gpt-4o ───────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  museums_info = web_search(query="top museums in the world 2023  
  visitor count")                                                 
  print(museums_info)                                             
 ────────────────────────────────────────────────────────────────

Execution logs:
[{'title': 'The Most Visited Museums in the World - 
worldstatistics.net', 'url': 
'https://worldstatistics.net/the-most-visited-museums-in-the-world
/', 'content': 'The Louvre in Paris, France, is the world’s most 
visited museum, with 8.9 million visitors in 2023. Museums in 
major cities like London, Paris, and Washington, D.C. tend to 
dominate the list of the world’s most visited attractions. Major 
cities like London, Paris, and Washington, D.C. host the world’s 
most visited museums, as noted by the first source.1 A lot of the 
most popular museums can be found in North America and Europe. The
most visited museums are mainly in big cities in North America and
Europe, says the first source.1 For example, in 2023, the Louvre 
in Paris saw 8,900,000 visitors,1 and the Vatican Museums had 
6,800,000 visitors.1 Also, the British Museum in London welcomed 
5,820,860 visitors,1 and New York City’s Metropolitan Museum of 
Art saw 5,400,000 guests.1', 'score': 0.9291904, 'raw_content': 
None}, {'title': 'List of most-visited museums - Wikipedia', 
'url': 
'https://en.wikipedia.org/wiki/List_of_most-visited_museums', 
'content': 'List of most-visited museums - Wikipedia National 
Museum of Natural History, Washington D.C., United States | 
Metropolitan Museum of Art | 5,400,000 (2023)[6] | New York City |
\xa0United States | | Museum of Modern Art | 2,839,509 (2023)[25] 
| New York City | \xa0United States | This list includes art 
museums, cultural museums, history museums, natural history 
museums, and science museums with an attendance of over 1,250,000 
people in 2022, but does not include archaeological sites, 
historical monuments, or most palace museums. List of most-visited
art museums List of most visited museums in the United Kingdom 
List of most-visited museums in the United States List of 
most-visited museums in France Retrieved from 
"https://en.wikipedia.org/w/index.php?title=List_of_most-visited_m
useums&oldid=1272662378" Lists of art museums and galleries Lists 
of museums', 'score': 0.8834878, 'raw_content': None}, {'title': 
'The 100 most popular art museums in the world ... - The Art 
Newspaper', 'url': 
'https://www.theartnewspaper.com/2024/03/26/the-100-most-popular-a
rt-museums-in-the-world-2023', 'content': 'Every year since the 
Covid-19 pandemic forced art museums around the world to shut 
their doors, we have been asking the question: when will visitor 
numbers get back to where they were before? Elsewhere in New York,
the Museum of Modern Art (MoMA) had more than 2.8 million 
visitors, 2% up on 2018, which was its last full year of opening 
before the pandemic. In Germany the Humboldt Forum is going from 
strength to strength since opening three years ago, with 1.7 
million visitors in 2023 making it the country’s most visited art 
museum. The Nasjonalmuseet in Oslo welcomed an impressive 1.2 
million visitors during its first full year of opening, becoming 
the most popular art museum in Scandinavia.', 'score': 0.875031, 
'raw_content': None}, {'title': 'List of most-visited art museums 
- Wikipedia', 'url': 
'https://en.wikipedia.org/wiki/List_of_most-visited_art_museums', 
'content': 'List of most-visited art museums - Wikipedia List of 
most-visited art museums | (26) | National Museum of Modern and 
Contemporary Art (Seoul branch) |  Seoul | 2,021,771 (2023)[13] | 
| | (50) | Kelvingrove Art Gallery and Museum |  Glasgow | 
1,283,882 (2023)[13] |  | | (55) | National Museum of Art, 
Architecture and Design |  Oslo | 1,246,000 (2023)[13] |  | | (80)
| National Museum of Modern Art (MOMAT) |  Tokyo | 883,596 
(2023)[13] |  | | (86) | Museum of Fine Arts |  Budapest | 821,491
(2023)[13] |  | List of art museums List of largest art museums 
Art museum The arts Art museums Retrieved from 
"https://en.wikipedia.org/w/index.php?title=List_of_most-visited_a
rt_museums&oldid=1282159123" Lists of art museums and galleries 
List of most-visited art museums', 'score': 0.8512743, 
'raw_content

[Step 1: Duration 2.65 seconds| Input tokens: 2,097 | Output 
tokens: 114]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
#modified code to handle case where agent doesn't return a python list.
import pandas as pd

try:
    display(pd.DataFrame(result))
except Exception as e:
    print("Could not display as DataFrame:", e)
    print(result)

,name,location,visitors,temp_july_f
0,Louvre,"Paris, France",8.90,63°F - 75°F
1,Vatican Museums,Vatican City,6.80,around 77°F
2,British Museum,"London, UK",5.82,66°F


## Create a multi-agent researcher
### Start with web search agent

In [13]:
web_agent = CodeAgent(
    model=OpenAIServerModel(
        "gpt-4o",
        max_completion_tokens=8096,
    ),
    tools=[web_search, VisitWebpageTool()],
    max_steps=10,
    name="web_agent",
    description="Runs web searches for you."
)
web_agent.logger.console.width=66

### Setup manager agent

In [14]:
from smolagents import Tool
from typing import Any
from smolagents.utils import make_image_url, encode_image_base64

def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel(
        "gpt-4o",
    )
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
        "Also, any run that invents numbers should fail."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=OpenAIServerModel(
        "gpt-4o",
        max_tokens=8096,
    ),
    tools=[],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "plotly.express",  #added
        "plotly.express.colors",  #added
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)
manager_agent.logger.console.width=66

In [15]:
manager_agent.visualize()

CodeAgent | gpt-4o
├── ✅ Authorized imports: ['geopandas', 'plotly', 
│   'plotly.express', 'plotly.express.colors', 'shapely', 'json', 
│   'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description          ┃ Arguments            ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final     │ answer (`any`): The  │
│   │              │ answer to the given  │ final answer to the  │
│   │              │ problem.             │ problem              │
│   └──────────────┴──────────────────────┴──────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gpt-4o
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Runs web searches for you.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description      ┃ Arguments       ┃
            ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
            │ web_search    │ Searches the web │ query           │
            │               │ for your query.  │ (`string`):     │
            │               │                  │ Your query      │
            │ visit_webpage │ Visits a webpage │ url (`string`): │
            │               │ at the given url │ The url of the  │
            │               │ and reads its    │ webpage to      │
            │               │ content as a     │ visit.          │
            │               │ markdown string. │                 │
            │               │ Use this to      │                 │
            │               │ browse webpages. │                 │
            │ final_answer  │ Provides a final │ answer (`any`): │
            │               │ answer to the    │ The final       │
            │               │ given problem.   │ answer to the   │
            │               │                  │ problem         │
            └───────────────┴──────────────────┴─────────────────┘

In [16]:
# added this code, remove plot file if exits.
import os
os.path.exists("saved_map.png") and os.remove("saved_map.png")

In [ ]:
manager_agent.run(f"""
{request_museums}

Then make me a spatial map of the world using px.scatter_map, with the biggest museums 
represented as scatter points of size depending on visitor count and color depending 
on the average temperature in July.
Save the map to saved_map.png, then return it!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour",
     color_continuous_scale=px.colors.sequential.Magma_r, size_max=15, zoom=1)
fig.show()
final_answer(fig)

Do not invent any numbers! You must only use numbers sourced from the internet.
""")

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Could you give me a sorted list of the top 3 museums in the    │
│ world in 2023,                                                 │
│ along with their visitor count (in millions) that year, and    │
│ the approximate daily temperature                              │
│ in July at their location ?                                    │
│                                                                │
│ Then make me a spatial map of the world using px.scatter_map,  │
│ with the biggest museums                                       │
│ represented as scatter points of size depending on visitor     │
│ count and color depending                                      │
│ on the average temperature in July.                            │
│ Save the map to saved_map.png, then return it!                 │
│                                                                │
│ Here's an example of how to plot and return a map:             │
│ import plotly.express as px                                    │
│ df = px.data.carshare()                                        │
│ fig = px.scatter_map(df, lat="centroid_lat",                   │
│ lon="centroid_lon", text="name", color="peak_hour",            │
│      color_continuous_scale=px.colors.sequential.Magma_r,      │
│ size_max=15, zoom=1)                                           │
│ fig.show()                                                     │
│ final_answer(fig)                                              │
│                                                                │
│ Do not invent any numbers! You must only use numbers sourced   │
│ from the internet.                                             │
│                                                                │
╰─ OpenAIServerModel - gpt-4o ───────────────────────────────────╯

────────────────────────── Initial plan ──────────────────────────
Here are the facts I know and the plan of action that I will 
follow to solve the task:
```
## Facts survey

### 1.1. Facts given in the task
- No specific facts are directly given except a brief reference to
using px.scatter_map for plotting.

### 1.2. Facts to look up
1. The top 3 museums in the world in 2023:
   - Source: A reliable and updated internet source such as a 
reputable ranking agency, news article, or cultural report.

2. Visitor count for each of these top 3 museums in 2023 (in 
millions):
   - Source: Museums' official websites or publications, cultural 
authority reports, or tourism statistics websites.

3. Approximate daily temperature in July at the location of these 
museums:
   - Source: Weather database websites such as the National 
Weather Service, Weather.com, or climate data websites.

4. The geographic coordinates (latitude and longitude) of each 
museum:
   - Source: Museums' official sites, Google Maps, or geographic 
database websites.

### 1.3. Facts to derive
- From the information gathered:
  1. Using the visitor counts, determine the relative size of 
scatter points in the map visualization.
  2. Using the daily temperature, determine the color coding in 
the map visualization for each museum.
  3. Combine the above data into a data frame suitable for input 
into px.scatter_map.

## Plan

1. Task the web_agent to find the top 3 museums in the world in 
2023 and retrieve their names.

2. Task the web_agent to gather visitor data for each of the top 3
museums in 2023 in millions.

3. Task the web_agent to find the average daily temperature in 
July for each museum's location.

4. Task the web_agent to find the geographic coordinates (latitude
and longitude) for the locations of each of these top 3 museums.

5. Compile and organize the data collected from the web_agent into
a structured data frame format. The data frame should contain the 
museum's name, visitor count, average daily temperature in July, 
and geographic coordinates.

6. Use the plotly express library to create a scatter map using 
the data frame:
   - The size of each scatter point should represent the visitor 
count.
   - The color of each scatter point should represent the average 
daily temperature in July.

7. Save the generated map to 'saved_map.png'.

8. Return the saved map using final_answer(fig).


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Sure, I'll start by tasking the web_agent to find the names of the
top 3 museums in the world in 2023.                               
                                                                  
Code:                                                             
```py                                                             
top_museums_task = """                                            
Please search for the top 3 museums in the world in 2023. Provide 
their names along with their global ranking based on any credible 
source such as a reputable ranking agency, news article, or       
cultural report.                                                  
"""                                                               
top_museums = web_agent(task=top_museums_task)                    
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  top_museums_task = """                                          
  Please search for the top 3 museums in the world in 2023.       
  Provide their names along with their global ranking based on    
  any credible source such as a reputable ranking agency, news    
  article, or cultural report.                                    
  """                                                             
  top_museums = web_agent(task=top_museums_task)                  
 ────────────────────────────────────────────────────────────────

...

Output message of the LLM: ───────────────────────────────────────
We now have the average July temperatures for the locations of the
top 3 museums:                                                    
                                                                  
1. **The Louvre, Paris, France**:                                 
   - Average daily high: 30.5°C                                   
   - Average daily low: 17.9°C                                    
                                                                  
2. **Vatican Museums, Vatican City**:                             
   - Average daily high: 31°C                                     
   - Average daily low: 21°C                                      
                                                                  
3. **British Museum, London, United Kingdom**:                    
   - Average daily high: 23.5°C                                   
   - Average daily low: 18°C                                      
                                                                  
Next, I'll task the web_agent to find the geographic coordinates  
for these museums.                                                
                                                                  
Code:                                                             
```py                                                             
coordinates_task = """                                            
Please find the geographic coordinates (latitude and longitude)   
for the following museums:                                        
1. The Louvre, Paris, France                                      
2. Vatican Museums, Vatican City                                  
3. British Museum, London, United Kingdom                         
"""                                                               
coordinates_data = web_agent(task=coordinates_task)               
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  coordinates_task = """                                          
  Please find the geographic coordinates (latitude and            
  longitude) for the following museums:                           
  1. The Louvre, Paris, France                                    
  2. Vatican Museums, Vatican City                                
  3. British Museum, London, United Kingdom                       
  """                                                             
  coordinates_data = web_agent(task=coordinates_task)             
 ────────────────────────────────────────────────────────────────

╭───────────────────── New run - web_agent ──────────────────────╮
│                                                                │
│ You're a helpful agent named 'web_agent'.                      │
│ You have been submitted this task by your manager.             │
│ ---                                                            │
│ Task:                                                          │
│                                                                │
│ Please find the geographic coordinates (latitude and           │
│ longitude) for the following museums:                          │
│ 1. The Louvre, Paris, France                                   │
│ 2. Vatican Museums, Vatican City                               │
│ 3. British Museum, London, United Kingdom                      │
│                                                                │
│ ---                                                            │
│ You're helping your manager solve a wider task: so make sure   │
│ to not provide a one-line answer, but give as much information │
│ as possible to give them a clear understanding of the answer.  │
│                                                                │
│ Your final_answer WILL HAVE to contain these parts:            │
│ ### 1. Task outcome (short version):                           │
│ ### 2. Task outcome (extremely detailed version):              │
│ ### 3. Additional context (if relevant):                       │
│                                                                │
│ Put all these in your final_answer tool, everything that you   │
│ do not pass as an argument to final_answer will be lost.       │
│ And even if your task resolution is not successful, please     │
│ return as much context as possible, so that your manager can   │
│ act upon this feedback.                                        │
│                                                                │
╰─ OpenAIServerModel - gpt-4o ───────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  louvre_coordinates_search = web_search(query="The Louvre Paris  
  France geographic coordinates")                                 
  print(louvre_coordinates_search)                                
 ────────────────────────────────────────────────────────────────

> Note, the map will look a bit different as we are only searching for 3 locations.

In [ ]:
fig = manager_agent.python_executor.state["fig"]
fig.update_layout(width=700, height=700)
fig.show()